# TINA Djara Olivier

# Exercice 1

In [1]:
import nltk
import requests
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.wsd import lesk
from googletrans import Translator

In [2]:
# Assurez-vous d'avoir téléchargé WordNet
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/olivier/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### 1. Constituer le contexte du document en recuperant tous les termes sigfificatifs

In [3]:
def constituer_contexte(texte):
    # Tokenisation des mots
    mots = word_tokenize(texte)
    
    # Suppression des mots vides (stop words)
    mots = [mot.lower() for mot in mots if mot.isalnum() and mot.lower() not in stopwords.words('english')]
    
    # Identification des noms et des entités importantes (sujet, verbe, adjectif, etc.)
    termes_significatifs = [mot for mot, pos in pos_tag(mots) if pos.startswith('N') or pos.startswith('V') or pos.startswith('J')]

    return termes_significatifs

In [4]:
texte_anglais = "Machine learning is a subfield of artificial intelligence that focuses on the development of algorithms and statistical models that computers use to perform a specific task without using explicit instructions. Natural Language Processing (NLP) is an essential component of machine learning, enabling computers to understand, interpret, and generate human-like language."

contexte = constituer_contexte(texte_anglais)
print(contexte)

['machine', 'learning', 'subfield', 'artificial', 'intelligence', 'focuses', 'development', 'statistical', 'models', 'computers', 'use', 'perform', 'specific', 'task', 'using', 'explicit', 'instructions', 'natural', 'language', 'processing', 'nlp', 'essential', 'component', 'machine', 'learning', 'enabling', 'computers', 'understand', 'interpret', 'generate', 'language']


### 2. Découper le texte en des phrases simples et recuperer les tags de leurs mots.

In [5]:
def decouper_et_taguer(texte):
    # Tokenisation en phrases
    phrases = nltk.sent_tokenize(texte)

    # Tokenisation des mots et étiquetage grammatical pour chaque phrase
    phrases_taguees = [nltk.pos_tag(nltk.word_tokenize(phrase)) for phrase in phrases]

    return phrases_taguees

In [6]:
resultat = decouper_et_taguer(texte_anglais)

# Affichage du résultat
for i, phrase_taguee in enumerate(resultat, 1):
    print(f"Phrase {i}: {phrase_taguee}")

Phrase 1: [('Machine', 'NN'), ('learning', 'NN'), ('is', 'VBZ'), ('a', 'DT'), ('subfield', 'NN'), ('of', 'IN'), ('artificial', 'JJ'), ('intelligence', 'NN'), ('that', 'WDT'), ('focuses', 'VBZ'), ('on', 'IN'), ('the', 'DT'), ('development', 'NN'), ('of', 'IN'), ('algorithms', 'NN'), ('and', 'CC'), ('statistical', 'JJ'), ('models', 'NNS'), ('that', 'IN'), ('computers', 'NNS'), ('use', 'VBP'), ('to', 'TO'), ('perform', 'VB'), ('a', 'DT'), ('specific', 'JJ'), ('task', 'NN'), ('without', 'IN'), ('using', 'VBG'), ('explicit', 'JJ'), ('instructions', 'NNS'), ('.', '.')]
Phrase 2: [('Natural', 'JJ'), ('Language', 'NNP'), ('Processing', 'NNP'), ('(', '('), ('NLP', 'NNP'), (')', ')'), ('is', 'VBZ'), ('an', 'DT'), ('essential', 'JJ'), ('component', 'NN'), ('of', 'IN'), ('machine', 'NN'), ('learning', 'NN'), (',', ','), ('enabling', 'VBG'), ('computers', 'NNS'), ('to', 'TO'), ('understand', 'VB'), (',', ','), ('interpret', 'VB'), (',', ','), ('and', 'CC'), ('generate', 'VB'), ('human-like', 'JJ'),

### 3. Pour chaque phrase récuperer le sens exacte de chaque terme en se basant sur leurs Tags et leur contexts

In [7]:
def recuperer_sens_exact(phrases_taguees):
    resultats = []

    for phrase_taguee in phrases_taguees:
        phrase_resultat = []

        for mot, tag in phrase_taguee:

            try:
                sens = lesk(phrase_taguee, mot)
                if sens:
                    definition = wordnet.synset(sens.name()).definition()
                else:
                    definition = "Aucun sens trouvé"
            except:
                definition = "Erreur lors de la récupération du sens"

            phrase_resultat.append((mot, tag, definition))

        resultats.append(phrase_resultat)

    return resultats

In [8]:
phrases_taguees = decouper_et_taguer(texte_anglais)
resultats = recuperer_sens_exact(phrases_taguees)

# Affichage du résultat
for i, phrase_resultat in enumerate(resultats, 1):
    print(f"Résultat pour la phrase {i}: {phrase_resultat}")

Résultat pour la phrase 1: [('Machine', 'NN', 'make by machinery'), ('learning', 'NN', 'impart skills or knowledge to'), ('is', 'VBZ', 'have an existence, be extant'), ('a', 'DT', 'any of several fat-soluble vitamins essential for normal vision; prevents night blindness or inflammation or dryness of the eyes'), ('subfield', 'NN', 'Aucun sens trouvé'), ('of', 'IN', 'Aucun sens trouvé'), ('artificial', 'JJ', 'not arising from natural growth or characterized by vital processes'), ('intelligence', 'NN', 'information about recent and important events'), ('that', 'WDT', 'Aucun sens trouvé'), ('focuses', 'VBZ', 'special emphasis attached to something'), ('on', 'IN', 'in a state required for something to function or be effective'), ('the', 'DT', 'Aucun sens trouvé'), ('development', 'NN', '(biology) the process of an individual organism growing organically; a purely biological unfolding of events involved in an organism changing gradually from a simple to a more complex level'), ('of', 'IN', '

### 4. Récuperer les termes correspendant en francais

In [9]:
def traduire_en_francais(mot):
    try:
        url = f"https://api.mymemory.translated.net/get?q={mot}&langpair=en|fr"
        response = requests.get(url)
        resultat_traduction = response.json()
        
        if "responseData" in resultat_traduction and "translatedText" in resultat_traduction["responseData"]:
            terme_francais = resultat_traduction["responseData"]["translatedText"]
            return terme_francais
        else:
            raise Exception("Erreur de traduction")
    except Exception as e:
        print(f"Erreur lors de la traduction : {e}")
        return "Erreur de traduction"

### 5. Pour chaque phrase afficher à l'utilisateur les propostions de traduction pour les nom, les adjectifs et les verbes

In [12]:
def recuperer_sens_exact(phrases_taguees):
    resultats = []

    for phrase_taguee in phrases_taguees:
        phrase_resultat = []

        for mot, tag in phrase_taguee:
            # Utilisation de l'algorithme Lesk pour récupérer le sens du mot
            try:
                sens = lesk(phrase_taguee, mot)
                if sens:
                    definition = wordnet.synset(sens.name()).definition()
                else:
                    definition = "Aucun sens trouvé"
            except:
                definition = "Erreur lors de la récupération du sens"

            # Traduire le terme en français
            terme_francais = traduire_en_francais(mot)

            phrase_resultat.append((mot, tag, terme_francais, definition))

        resultats.append(phrase_resultat)

    return resultats

In [13]:
phrases_taguees = decouper_et_taguer(texte_anglais)
resultats = recuperer_sens_exact(phrases_taguees)

# Affichage des résultats avec propositions de traduction
for i, phrase_resultat in enumerate(resultats, 1):
    phrase_traduite = []

    for terme, tag, terme_francais, definition in phrase_resultat:
        # Afficher les propositions de traduction pour les noms, adjectifs et verbes
        if tag.startswith('N'):
            traductions_proposees = [traduire_en_francais(terme) for terme in [terme, definition] if terme]
        elif tag.startswith('J'):
            traductions_proposees = [traduire_en_francais(terme) for terme in [terme, definition] if terme]
        elif tag.startswith('V'):
            traductions_proposees = [traduire_en_francais(terme) for terme in [terme, definition] if terme]
        else:
            traductions_proposees = []

        phrase_traduite.append((terme_francais, tag, traductions_proposees, definition))

    print(f"Résultat pour la phrase {i}: {phrase_traduite}")

Résultat pour la phrase 1: [('Machine', 'NN', ['Machine', 'faire par machinerie'], 'make by machinery'), ('apprentissage', 'NN', ['apprentissage', 'transmettre des compétences ou des connaissances à'], 'impart skills or knowledge to'), ('is', 'VBZ', ['is', 'avoir une existence, être existant'], 'have an existence, be extant'), ('a', 'DT', [], 'any of several fat-soluble vitamins essential for normal vision; prevents night blindness or inflammation or dryness of the eyes'), ('sous-champ', 'NN', ['sous-champ', 'Aucun sens trouvé'], 'Aucun sens trouvé'), ('de', 'IN', [], 'Aucun sens trouvé'), ('gazon artificiel', 'JJ', ['gazon artificiel', "ne résultant pas d'une croissance naturelle ou caractérisée par des processus vitaux"], 'not arising from natural growth or characterized by vital processes'), ('renseignement;', 'NN', ['renseignement;', 'informations sur les événements récents et importants'], 'information about recent and important events'), ('cette', 'WDT', [], 'Aucun sens trouvé'),

# Exercice 2 : dection de plagiarisme

In [43]:
import os
import Levenshtein
import pandas as pd

## Texte original

In [17]:
dossier = "plagitat_corpus/plagitat_corpus/original"
text_wikipedia = []

fichiers_dans_dossier = os.listdir(dossier)

for nom_fichier in fichiers_dans_dossier:
    chemin_complet = os.path.join(dossier, nom_fichier)
    if os.path.isfile(chemin_complet) and nom_fichier.endswith(".txt"):
        with open(chemin_complet, 'r', encoding='utf-8') as fichier:
            contenu = fichier.read()
            text_wikipedia.append(contenu)

## Textes des étudiants

In [27]:
dossier1 = "plagitat_corpus/plagitat_corpus/PA"
PA = []

fichiers_dans_dossier = os.listdir(dossier1)

for nom_fichier in fichiers_dans_dossier:
    chemin_complet = os.path.join(dossier1, nom_fichier)
    if os.path.isfile(chemin_complet) and nom_fichier.endswith(".txt"):
        with open(chemin_complet, 'r', encoding='utf-8') as fichier:
            contenu = fichier.read()
            PA.append(contenu)


dossier2 = "plagitat_corpus/plagitat_corpus/PB"
PB = []

fichiers_dans_dossier = os.listdir(dossier2)

for nom_fichier in fichiers_dans_dossier:
    chemin_complet = os.path.join(dossier2, nom_fichier)
    if os.path.isfile(chemin_complet) and nom_fichier.endswith(".txt"):
        with open(chemin_complet, 'r', encoding='utf-8') as fichier:
            contenu = fichier.read()
            PB.append(contenu)


dossier3 = "plagitat_corpus/plagitat_corpus/PC"
PC = []

fichiers_dans_dossier = os.listdir(dossier3)

for nom_fichier in fichiers_dans_dossier:
    chemin_complet = os.path.join(dossier3, nom_fichier)
    if os.path.isfile(chemin_complet) and nom_fichier.endswith(".txt"):
        with open(chemin_complet, 'r', encoding='utf-8') as fichier:
            contenu = fichier.read()
            PC.append(contenu)


dossier4 = "plagitat_corpus/plagitat_corpus/PD"
PD = []

fichiers_dans_dossier = os.listdir(dossier4)

for nom_fichier in fichiers_dans_dossier:
    chemin_complet = os.path.join(dossier4, nom_fichier)
    if os.path.isfile(chemin_complet) and nom_fichier.endswith(".txt"):
        with open(chemin_complet, 'r', encoding='utf-8') as fichier:
            contenu = fichier.read()
            PD.append(contenu)


dossier5 = "plagitat_corpus/plagitat_corpus/PE"
PE = []

fichiers_dans_dossier = os.listdir(dossier5)

for nom_fichier in fichiers_dans_dossier:
    chemin_complet = os.path.join(dossier5, nom_fichier)
    if os.path.isfile(chemin_complet) and nom_fichier.endswith(".txt"):
        with open(chemin_complet, 'r', encoding='utf-8') as fichier:
            contenu = fichier.read()

            PE.append(contenu)

### Calcul de similarité pour le texte original et le texte des 5 étudiants

In [36]:
sim1, sim2, sim3, sim4, sim5 = [], [], [], [], []

for i in range(len(text_wikipedia)):
    distance = Levenshtein.distance(text_wikipedia[i], PA[i])
    similarity = 1 - (distance / max(len(text_wikipedia[i]), len(PA[i])))
    sim1.append(similarity)

    distance = Levenshtein.distance(text_wikipedia[i], PB[i])
    similarity = 1 - (distance / max(len(text_wikipedia[i]), len(PB[i])))
    sim2.append(similarity)

    distance = Levenshtein.distance(text_wikipedia[i], PC[i])
    similarity = 1 - (distance / max(len(text_wikipedia[i]), len(PC[i])))
    sim3.append(similarity)

    distance = Levenshtein.distance(text_wikipedia[i], PD[i])
    similarity = 1 - (distance / max(len(text_wikipedia[i]), len(PD[i])))
    sim4.append(similarity)

    distance = Levenshtein.distance(text_wikipedia[i], PE[i])
    similarity = 1 - (distance / max(len(text_wikipedia[i]), len(PE[i])))
    sim5.append(similarity)

### Scores de similarités

In [46]:
# Créer un dictionnaire avec les données
data = {
    'sim1': sim1,
    'sim2': sim2,
    'sim3': sim3,
    'sim4': sim4,
    'sim5': sim5,
}

# Créer le DataFrame
df = pd.DataFrame(data)

# Afficher le DataFrame
df

,sim1,sim2,sim3,sim4,sim5
0,0.254060,0.216899,0.484023,0.177580,0.157674
1,0.243553,0.222222,0.216810,0.154091,0.125119
2,0.752964,0.250847,0.270092,0.123188,0.232543
3,0.262840,0.273917,0.251259,0.472810,0.209970
4,0.335700,0.248870,0.223370,0.214977,0.269206
